In [2]:
from pathlib import Path
import napari
from utils import list_images, read_image

In [3]:
# Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
directory_path = Path("./raw_data/test_data")

# Iterate through the .czi and .nd2 files in the directory
images = list_images(directory_path)

images

['raw_data\\test_data\\WellF03_ChannelSD_AF647,SD_RFP,SD_GFP,DAPI,BF-SD_Seq0069.nd2',
 'raw_data\\test_data\\WellG09_ChannelSD_AF647,SD_RFP,SD_GFP,DAPI,BF-SD_Seq0080.nd2']

In [9]:
# Explore a different image to crop (0 defines the first image in the directory)
image = images[1]

# Image size reduction (downsampling) to improve processing times (slicing, not lossless compression)
slicing_factor_xy = None # Use 2 or 4 for downsampling in xy (None for lossless)

# Read image, apply slicing if needed and return filename and img as a np array
img, filename = read_image(image, slicing_factor_xy)

# Show image in Napari to define ROI
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(img)



Image analyzed: WellG09_ChannelSD_AF647,SD_RFP,SD_GFP,DAPI,BF-SD_Seq0080
Original Array shape: (5, 5032, 5032)
Compressed Array shape: (5, 5032, 5032)


<Image layer 'img' at 0x210bd18f670>

In [5]:
import apoc

In [6]:
img.shape

(5, 5032, 5032)

In [11]:
# Explore a different image to crop (0 defines the first image in the directory)
image = images[1]

# Image size reduction (downsampling) to improve processing times (slicing, not lossless compression)
slicing_factor_xy = None # Use 2 or 4 for downsampling in xy (None for lossless)

# Read image, apply slicing if needed and return filename and img as a np array
img, filename = read_image(image, slicing_factor_xy)

viewer = napari.Viewer(ndisplay=2)
viewer.add_image(img[1])



Image analyzed: WellG09_ChannelSD_AF647,SD_RFP,SD_GFP,DAPI,BF-SD_Seq0080
Original Array shape: (5, 5032, 5032)
Compressed Array shape: (5, 5032, 5032)


<Image layer 'Image' at 0x1f733cf2da0>

In [14]:
image_folder = "./raw_data/SD_RFP_spot_detection_training/images"
annotation_folder = "./raw_data/SD_RFP_spot_detection_training/annotations"

In [39]:
# Setup classifer and where it should be saved
cl_filename = "./SD_RFP_spot_segmenter.cl"
apoc.erase_classifier(cl_filename)
segmenter = apoc.ObjectSegmenter(opencl_filename=cl_filename)

# Setup feature set used for training
features = apoc.PredefinedFeatureSet.object_size_1_to_5_px.value #TODO: Check

# Train classifier on folders
apoc.train_classifier_from_image_folders(
    segmenter, 
    features, 
    image = image_folder, # Grayscale processed images, no RGB
    ground_truth = annotation_folder)

In [7]:
cl_filename = "./SD_RFP_spot_segmenter.cl"
viewer = napari.Viewer(ndisplay=2)
segmenter = apoc.ObjectSegmenter(opencl_filename=cl_filename)

In [28]:
image = images[0]

# Read image, apply slicing if needed and return filename and img as a np array
img, filename = read_image(image, slicing_factor_xy)

viewer = napari.Viewer(ndisplay=2)

predicted_labels = segmenter.predict(img[1])
viewer.add_image(img[1])
viewer.add_labels(predicted_labels)



Image analyzed: WellF03_ChannelSD_AF647,SD_RFP,SD_GFP,DAPI,BF-SD_Seq0069
Original Array shape: (5, 5032, 5032)
Compressed Array shape: (5, 5032, 5032)


<Labels layer 'predicted_labels' at 0x2110a1442b0>

In [26]:
import tifffile

In [27]:
annotations = tifffile.imread("./raw_data/SD_RFP_spot_detection_training/annotations/WellG09_SD_RFP.tiff")
viewer.add_labels(annotations)

<Labels layer 'annotations' at 0x1f684eb3fd0>

In [12]:
from spotiflow.model import Spotiflow

c:\Users\adiez_cmic\miniforge3\envs\brain_nuc_stardist\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
model = Spotiflow.from_pretrained("general")

INFO:spotiflow.model.spotiflow:Loading pretrained model: general


In [29]:
points, details = model.predict(img[1], subpix=True)

INFO:spotiflow.model.spotiflow:Will use device: cpu
INFO:spotiflow.model.spotiflow:Predicting with prob_thresh = [0.6], min_distance = 1
INFO:spotiflow.model.spotiflow:Peak detection mode: fast
INFO:spotiflow.model.spotiflow:Image shape (5032, 5032)
INFO:spotiflow.model.spotiflow:Predicting with (3, 3) tiles
INFO:spotiflow.model.spotiflow:Normalizing...
INFO:spotiflow.model.spotiflow:Padding to shape (5040, 5040, 1)


Predicting tiles: 100%|██████████| 9/9 [00:24<00:00,  2.77s/it]

INFO:spotiflow.model.spotiflow:Found 211 spots


In [32]:
viewer.add_image(details.heatmap, colormap="viridis")

<Image layer 'Image [1]' at 0x2110a3f3640>